# Buoy Based Model- Point Conception Transmitivity

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.structured import *
from fastai.column_data import *
from fasterai.files import *
from fasterai.structured import *
from pathlib import Path
from itertools import repeat
import re

In [3]:
from IPython.display import HTML
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

np.set_printoptions(threshold=50, edgeitems=20)
PATH=Path('data/divevis')

In [4]:
class ObsTableInfo():
    def __init__(self, path: Path):
        self.table = pd.read_csv(path, low_memory=False)
        self.station = path.stem
        self.year = path.parent.name

## Load Raw datasets

In [5]:
obs_table_paths = find_files_recursively(PATH/'obs', ('csv'))

In [6]:
obs_tables_infos = [ObsTableInfo(Path(path)) for path in obs_table_paths]

The following returns summarized aggregate information to each table accross each field.

## Data Cleaning / Feature Engineering

As a structured data problem, we necessarily have to go through all the cleaning and feature engineering, even though we're using a neural network.

#### Drop Unecessary Columns

In [7]:
obs_drop_columns=['AA1','AA2','AA3','AB1','AD1','AE1','AH1','AH2','AH3','AH4','AH5','AH6','AI1','AI2','AI3','AI4','AI5','AI6',
         'AJ1','AK1','AM1','AN1','AT1','AT2','AT3','AT4','AU1','AU2','AU3','AW1','AW2','AW3','AX1','GA1','GA2','GA3','GD1',
         'GD2','GD3','GF1','KA1','KA2','KB1','KB2','KB3','KC1','KC2','KD1','KD2','KE1','KG1','KG2','MA1','MD1','MF1',
         'MG1','MH1','MK1','MV1','MW1','MW2','MW3','OC1','OD1','OE1','OE2','OE3','RH1','RH2','RH3','EQD','QUALITY_CONTROL',
         'CALL_SIGN','REPORT_TYPE','NAME','ELEVATION','LONGITUDE','LATITUDE','SOURCE', 'SLP', 'VIS','TMP', 'CIG','DEW']

for table_info in obs_tables_infos:
    table = table_info.table
    table.drop(columns=obs_drop_columns,inplace=True, errors='ignore')

#### Normalize Dates/Times for Later Joins

In [8]:
for table_info in obs_tables_infos:
    table = table_info.table
    name='DATE'
    table[name]=pd.to_datetime(table[name])
    table[name] = table[name].apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour, 0))

#### Get Rid of Non-Metar Rows

In [10]:
for table_info in obs_tables_infos:
    table = table_info.table
    table = table[table['REM'].notnull()]
    table = table[table['REM'].notna()]
    table = table[table['REM'].str.startswith('MET')]
    table_info.table=table

#### Decompose Wind into Separate Columns

In [20]:
#MET09701/04/12 13:54:02 METAR KBFL 042154Z 33010KT 5SM HZ CLR 21/06 A3026 RMK AO2 SLP244 T02110061 (RN)
wnd_pat = re.compile(r'(.*?\s+)(\d{3})(\d{2})(KT\s+.*?)', re.IGNORECASE)

for table_info in obs_tables_infos:
    table = table_info.table
    source='REM'
    table['WND_DIR'] = table[source].apply(lambda wnd: wnd_pat.match(str(wnd)).group(2) if wnd_pat.match(str(wnd)) is not None else 999)
    table['WND_SP'] = table[source].apply(lambda wnd: wnd_pat.match(str(wnd)).group(3) if wnd_pat.match(str(wnd)) is not None else 0)

#### Extract Temperatures/Dew Points

In [12]:
temp_dew_pat = re.compile(r'(.*?\s+)(\d{2})(/)(\d{2})(\s+.*?)', re.IGNORECASE)

for table_info in obs_tables_infos:
    table = table_info.table
    source='REM'
    table['TMP'] = table[source].apply(lambda temp_dew: temp_dew_pat.match(str(temp_dew)).group(2) if temp_dew_pat.match(str(temp_dew)) is not None else 99)
    table['DEW'] = table[source].apply(lambda temp_dew: temp_dew_pat.match(str(temp_dew)).group(4) if temp_dew_pat.match(str(temp_dew)) is not None else 99)

#### Extract Sea Level Pressure

In [13]:
slp_pat = re.compile(r'(.*?\s+)(SLP)(\d+)(\s+.*?)', re.IGNORECASE)

for table_info in obs_tables_infos:
    table = table_info.table
    source='REM'
    table['SLP'] = table[source].apply(lambda slp: slp_pat.match(str(slp)).group(3) if slp_pat.match(str(slp)) is not None else 999)

#### Extract Visibility

In [14]:
vis_pat = re.compile(r'(.*?\s+)(\d+)(SM)(\s+.*?)', re.IGNORECASE)
for table_info in obs_tables_infos:
    table = table_info.table
    source='REM'
    table_info.table['VIS'] = table[source].apply(lambda vis: vis_pat.match(str(vis)).group(2) if vis_pat.match(str(vis)) is not None else 999)

In [15]:
display(obs_tables_infos[0].table.head())

,STATION,DATE,WND,REM,WND_DIR,WND_SP,TMP,DEW,SLP,VIS
1,72274023160,2012-01-01 00:00:00,"999,9,C,0000,5",MET09512/31/11 17:53:03 METAR KTUS 010053Z 000...,000,00,19,01,177,10
2,72274023160,2012-01-01 01:00:00,"999,9,C,0000,5",MET09512/31/11 18:53:03 METAR KTUS 010153Z 000...,000,00,16,01,187,10
3,72274023160,2012-01-01 02:00:00,"170,5,N,0021,5",MET10112/31/11 19:53:03 METAR KTUS 010253Z 170...,170,04,15,01,194,10
4,72274023160,2012-01-01 03:00:00,"999,9,C,0000,5",MET09512/31/11 20:53:03 METAR KTUS 010353Z 000...,000,00,12,02,202,10
5,72274023160,2012-01-01 04:00:00,"140,5,N,0021,5",MET09512/31/11 21:53:03 METAR KTUS 010453Z 140...,140,04,11,02,203,10


In [21]:
display(DataFrameSummary(obs_tables_infos[0].table).summary())

,STATION,DATE,WND,REM,WND_DIR,WND_SP,TMP,DEW,SLP,VIS
count,9164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,7.2274e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,7.2274e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,7.2274e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,7.2274e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,7.2274e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,7.2274e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
counts,9164,9164,9164,9164,9164,9164,9164,9164,9164,9164
uniques,1,8781,634,9164,38,20,44,25,269,11


In [ ]:
icb1v12 = tables[0]

In [ ]:
len(icb1v12)

`join_df` is a function for joining tables on specific fields. By default, we'll be doing a left outer join of `right` on the `left` argument using the given fields for each table.

Pandas does joins using the `merge` method. The `suffixes` argument describes the naming convention for duplicate fields. We've elected to leave the duplicate field names on the left untouched, and append a "\_y" to those on the right.

In [ ]:
def join_df(left, right, left_on, right_on=None, suffix='_y'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, 
                      suffixes=("", suffix))

### Date Cleanup/Features

The following turns raw year, month and day columns into datetime

The following extracts particular date fields from a complete datetime for the purpose of constructing categoricals.

You should *always* consider this feature extraction step when working with date-time. Without expanding your date-time into these additional fields, you can't capture any trend/cyclical behavior as a function of time at any of these granularities. We'll add to every table with a date field.

In [ ]:
process_dates(dataframe=icb1v12, year_field='yrtrig', month_field='motrig', day_field='datrig', 
              datetime_field='datetime_trig')

process_dates(dataframe=icb1v12, year_field='yrterm', month_field='moterm', day_field='daterm', 
              datetime_field='datetime_term')

process_dates(dataframe=icb1v12, year_field='yrmedst', month_field='momedst', day_field='damedst', 
              datetime_field='datetime_medst')

process_dates(dataframe=icb1v12, year_field='yrmedend', month_field='momedend', day_field='damedend', 
              datetime_field='datetime_medend')

process_dates(dataframe=icb1v12, year_field='yrmedfin', month_field='momedfin', day_field='damedfin', 
              datetime_field='datetime_medfin')

display(icb1v12.head())

Drop useless generated date columns here

In [ ]:
useless_date_columns = ['datetime_medfinElapsed', 'datetime_medendElapsed', 
                        'datetime_medstElapsed', 'datetime_termElapsed', 'datetime_trigElapsed']
icb1v12.drop(useless_date_columns,1,inplace=True)

Convert booleans to 1s and 0s, so they can be used as categories later.

In [ ]:
boolean_columns = ['datetime_trigIs_month_end','datetime_trigIs_month_start','datetime_trigIs_quarter_end',
            'datetime_trigIs_quarter_start','datetime_trigIs_year_end','datetime_trigIs_year_start',
            'datetime_termIs_month_end','datetime_termIs_month_start','datetime_termIs_quarter_end',
            'datetime_termIs_quarter_start','datetime_termIs_year_end','datetime_termIs_year_start',
            'datetime_medstIs_month_end','datetime_medstIs_month_start','datetime_medstIs_quarter_end',
            'datetime_medstIs_quarter_start','datetime_medstIs_year_end','datetime_medstIs_year_start',
            'datetime_medendIs_month_end', 'datetime_medendIs_month_start',
            'datetime_medendIs_quarter_end','datetime_medendIs_quarter_start','datetime_medendIs_year_end',
            'datetime_medendIs_year_start','datetime_medfinIs_month_end',
            'datetime_medfinIs_month_start','datetime_medfinIs_quarter_end','datetime_medfinIs_quarter_start',
            'datetime_medfinIs_year_end','datetime_medfinIs_year_start']

icb1v12[boolean_columns] = (icb1v12[boolean_columns] == True).astype(int)

display(icb1v12.head())

Now we extract useful duration information

In [ ]:
def add_duration_from_datetimes(dataframe, begin_field, end_field, duration_field):
    dataframe[duration_field] = dataframe[end_field].subtract(dataframe[begin_field]).dt.days
    dataframe[duration_field] = np.where(dataframe[duration_field] < 0, 0, dataframe[duration_field]) 

In [ ]:
#TODO:  Should these durations be in days, months, or years?  In the latter two cases, perhaps make it categorical instead of
#continuous?

#Almost a duplicate of 'brexit', except this version calculates after date data discrepancies are accounted for above.  
#I don't think it'll hurt to have both.
add_duration_from_datetimes(dataframe=icb1v12, begin_field='datetime_trig', end_field='datetime_term', 
                            duration_field='event_duration')


#TODO:  The code book on mediation dates variables isn't consistent with what's in the CSV.  Which needs to be updated?  
#Based on the data, I'm assuming medend == Date of Meditation End Within Crises Period, 
#and medfin == Date of Meditation End Outside Crises Period

add_duration_from_datetimes(dataframe=icb1v12, begin_field='datetime_trig', end_field='datetime_medst', 
                            duration_field='mediation_start_delay_after_crises_start')


add_duration_from_datetimes(dataframe=icb1v12, begin_field='datetime_medst', end_field='datetime_medend', 
                            duration_field='mediation_duration_within_crisis')

add_duration_from_datetimes(dataframe=icb1v12, begin_field='datetime_medend', end_field='datetime_medfin', 
                            duration_field='mediation_duration_outside')

add_duration_from_datetimes(dataframe=icb1v12, begin_field='datetime_medst', end_field='datetime_medfin', 
                            duration_field='mediation_duration_total')

display(icb1v12[icb1v12['mediation_duration_within_crisis'] > 0])

### (TODO) Duration Between World Crises? Use function below

It is common when working with time series data to extract data that explains relationships across rows as opposed to columns, e.g.:
* Running averages
* Time until next event
* Time since last event

This is often difficult to do with most table manipulation frameworks, since they are designed to work with relationships across columns. As such, we've created a class to handle this type of data.

We'll define a function `get_elapsed` for cumulative counting across a sorted dataframe. Given a particular field `fld` to monitor, this function will start tracking time since the last occurrence of that field. When the field is seen again, the counter is set to zero.

Upon initialization, this will result in datetime na's until the field is encountered. This is reset every time a new store is seen. We'll see how to use this shortly.

In [ ]:
def get_elapsed(fld, pre):
    day1 = np.timedelta64(1, 'D')
    last_date = np.datetime64()
    last_store = 0
    res = []

    for s,v,d in zip(df.Store.values,df[fld].values, df.Date.values):
        if s != last_store:
            last_date = np.datetime64()
            last_store = s
        if v: last_date = d
        res.append(((d-last_date).astype('timedelta64[D]') / day1))
    df[pre+fld] = res

### Known Sources of NaN- Cleanup

Next we'll fill in missing values to avoid complications with `NA`'s. `NA` (not available) is how Pandas indicates missing values; many models have problems when missing values are present, so it's always important to think about how to deal with them. In these cases, we are picking an arbitrary *signal value* that doesn't otherwise appear in the data.

In [ ]:
def filter_to_rows_with_nans(dataframe):
    return dataframe[dataframe.isnull().any(axis=1)]

In [ ]:
#Since we're clasifying escalation/descalation, get rid of category#3 (unknown because of being recent).

icb1v12=icb1v12.drop(icb1v12[icb1v12.outesr == 3].index)


#This is a weird row- there's no data in it.
row_index_to_drop = icb1v12[icb1v12.crisno == 474].index
icb1v12=icb1v12.drop(row_index_to_drop)
icb1v12.usactor = icb1v12.usactor.fillna(1).astype(np.int32)
icb1v12.suinv = icb1v12.suinv.fillna(1).astype(np.int32)
icb1v12.suefct = icb1v12.suefct.fillna(1).astype(np.int32)
icb1v12.suefac = icb1v12.suefac.fillna(1).astype(np.int32)
icb1v12.supace = icb1v12.supace.fillna(1).astype(np.int32)
icb1v12.suactor = icb1v12.suactor.fillna(1).astype(np.int32)

icb1v12.powdissy = icb1v12.powdissy.fillna(1).astype(np.int32)

icb1v12.mediate = icb1v12.mediate.fillna(1).astype(np.int32)
icb1v12.mednum = icb1v12.mednum.fillna(1).astype(np.int32)
icb1v12.medwho = icb1v12.medwho.fillna(1).astype(np.int32)
icb1v12.medtime = icb1v12.medtime.fillna(1).astype(np.int32)
icb1v12.medgoal = icb1v12.medgoal.fillna(1).astype(np.int32)
icb1v12.medfacl = icb1v12.medfacl.fillna(1).astype(np.int32)
icb1v12.medform = icb1v12.medform.fillna(1).astype(np.int32)
icb1v12.medmanip = icb1v12.medmanip.fillna(1).astype(np.int32)
icb1v12.medstyle = icb1v12.medstyle.fillna(1).astype(np.int32)
icb1v12.medstefc = icb1v12.medstefc.fillna(1).astype(np.int32)
icb1v12.medefct = icb1v12.medefct.fillna(1).astype(np.int32)
icb1v12.medpace = icb1v12.medpace.fillna(1).astype(np.int32)

#TODO:  Perhaps default to 0 for dates < 1945, and 1 afterwards? RSO involvement didn't exist prior to that, basically-
#source: 1-37 of ICB1Codebook-v12.pdf
icb1v12.soract = icb1v12.soract.fillna(1).astype(np.int32)

#TODO:  Default on the next two are tough to determine.  
icb1v12.hetero = icb1v12.hetero.fillna(1).astype(np.int32)
icb1v12.issues = icb1v12.issues.fillna(1).astype(np.int32)

icb1v12.chacts = icb1v12.chacts.fillna(1).astype(np.int32)
icb1v12.chall = icb1v12.chall.fillna(1).astype(np.int32)
icb1v12.powch = icb1v12.powch.fillna(1).astype(np.int32)
icb1v12.rugach = icb1v12.rugach.fillna(1).astype(np.int32)

icb1v12.usefct = icb1v12.usefct.fillna(1).astype(np.int32)
icb1v12.usefac = icb1v12.usefac.fillna(1).astype(np.int32)
icb1v12.uspace = icb1v12.uspace.fillna(1).astype(np.int32)

#TODO:  Default is tough to determine. 
icb1v12.stressad = icb1v12.stressad.fillna(1).astype(np.int32)

display(filter_to_rows_with_nans(icb1v12))

## TODO: Additional geographic features? Using TRIGENT for triggering entity (country).  Actor level info useful here.  GDP/Per Capita Income/Other measures of wealth?  Government type? Military size?  Climate?  Biggest import/export?

## TODO: Rolling quanities?

## Save progress to file

It's usually a good idea to back up large tables of extracted / wrangled features before you join them onto another one, that way you can go back to it easily if you need to make changes to it.

In [ ]:
icb1v12.reset_index(inplace=True)
icb1v12.to_feather(f'{PATH}icb1v12')

We now have our final set of engineered features.

While these steps were explicitly outlined in the paper, these are all fairly typical feature engineering steps for dealing with time series data and are practical in any similar setting.

## Load progress from file

In [ ]:
icb1v12 = pd.read_feather(f'{PATH}icb1v12')

## Create features

In [ ]:
icb1v12.head().T.head(40)

Now that we've engineered all our features, we need to convert to input compatible with a neural network.

This includes converting categorical variables into contiguous integers or one-hot encodings, normalizing continuous features to standard normal, etc...

In [ ]:
cat_vars = ['break','trigent',
            #'yrtrig_clean','motrig_clean','datrig_clean','yrterm_clean','moterm_clean','daterm_clean', 
            'gravcr','crismg','cenviosy','sevviosy','viol','timvio','iwcmb', 'gpinv','gpinvtp','gpefcttp','gpefactp','gppacetp',
            'powinv','usinv','usefct','usefac','uspace','usactor','suinv','suefct','suefac','supace','suactor','soglact',
            'globorg','globactm','globefct','globefor','globefac','globpace','soract','regorg','regactmb','roefct','robody',
            'roefac','ropace','subout','forout','exsat', 'geostr','hetero','issues','chacts','chall','powch','rugach',
            'geog','geogrel','period','syslevsy','protrac','ethnic','ethconf', 'sourdt',
            #'mediate','mednum','medwho','medtime',
            #'yrmedst_clean','momedst_clean','damedst_clean','yrmedend_clean','momedend_clean','damedend_clean',
            #'yrmedfin_clean','momedfin_clean', 'damedfin_clean',
            #'medgoal','medfacl','medform','medmanip','medstyle',
            #'medstefc','medefct','medpace',
            'datetime_trigDayofweek',
            'datetime_trigIs_month_end','datetime_trigIs_month_start','datetime_trigIs_quarter_end',
            #'datetime_trigIs_quarter_start','datetime_trigIs_year_end','datetime_trigIs_year_start',
            'datetime_termDayofweek',
            #'datetime_termIs_month_end','datetime_termIs_month_start','datetime_termIs_quarter_end',
            #'datetime_termIs_quarter_start','datetime_termIs_year_end','datetime_termIs_year_start', 
            #'datetime_medstDayofweek',
            #'datetime_medstIs_month_end','datetime_medstIs_month_start','datetime_medstIs_quarter_end',
            #'datetime_medstIs_quarter_start','datetime_medstIs_year_end','datetime_medstIs_year_start',
            #'datetime_medendDayofweek','datetime_medendIs_month_end', 'datetime_medendIs_month_start',
            #'datetime_medendIs_quarter_end','datetime_medendIs_quarter_start','datetime_medendIs_year_end',
            #'datetime_medendIs_year_start','datetime_medfinDayofweek', 'datetime_medfinIs_month_end',
            #'datetime_medfinIs_month_start','datetime_medfinIs_quarter_end','datetime_medfinIs_quarter_start',
            #'datetime_medfinIs_year_end','datetime_medfinIs_year_start'
]

contin_vars = [
        'brexit','noactr','cractr', 'pcid', 'powdissy', 'stressad', 'event_duration',
        #'mediation_start_delay_after_crises_start',
        #'mediation_duration_within_crisis','mediation_duration_outside',
        #'mediation_duration_total'
]

n = len(icb1v12); n

In [ ]:
dep = 'outesr'
icb1v12 = icb1v12[cat_vars+contin_vars+[dep, 'datetime_trig']].copy()

In [ ]:
#TODO:  Not sure if this is the best default
icb1v12[dep] = icb1v12[dep].fillna(3).astype(np.int32)
icb1v12[dep] = icb1v12[dep]-1

for v in cat_vars:
    icb1v12[v] = icb1v12[v].astype('category').cat.as_ordered()

In [ ]:
apply_cats(icb1v12.copy(), icb1v12)

In [ ]:
for v in contin_vars:
    icb1v12[v] = icb1v12[v].fillna(0).astype('float32')

In [ ]:
icb1v12.set_index("datetime_trig", inplace=True)

We can now process our data...

In [ ]:
icb1v12.head(2)

In [ ]:
df, y, nas, mapper = proc_df(icb1v12, dep, do_scale=True)

In [ ]:
df.head(2)

In time series data, cross-validation is not random. Instead, our holdout data is generally the most recent data, as it would be in real application. This issue is discussed in detail in [this post](http://www.fast.ai/2017/11/13/validation-sets/) on our web site.

One approach is to take the last 25% of rows (sorted by date) as our validation set.

In [ ]:
train_ratio = 0.75
train_size = int(n * train_ratio); train_size
val_idx = list(range(train_size, len(df)))

## DL

We're ready to put together our models.

We can create a ModelData object directly from out data frame.

In [ ]:
md = ColumnarModelData.from_data_frame(PATH, val_idx, df, y.astype('int'), cat_flds=cat_vars, bs=8,test_df=None, 
                                       is_reg=False,is_multi=False)

Some categorical variables have a lot more levels than others. Store, in particular, has over a thousand!

In [ ]:
cat_sz = [(c, len(icb1v12[c].cat.categories)+1) for c in cat_vars]

In [ ]:
cat_sz

We use the *cardinality* of each variable (that is, its number of unique values) to decide how large to make its *embeddings*. Each level will be associated with a vector with length defined as below.

In [ ]:
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]

In [ ]:
emb_szs

In [ ]:
m = md.get_learner(emb_szs, n_cont=len(df.columns)-len(cat_vars),
        emb_drop=0.05, out_sz=3, szs=[1000,500], drops=[0.001,0.01],y_range=[0,1],
        use_bn=True, crit=F.cross_entropy)
m

In [ ]:
m.lr_find()

In [ ]:
m.sched.plot()

### Training

In [ ]:
lr = 1e-3
m.fit(lr, n_cycle=3, metrics=[accuracy], cycle_len=1, cycle_mult=2, wds=1e-5)